In [1]:
from scheduler_class import CourseScheduleExtractor
courses = ['EEE-313', 'EEE-321', 'EEE-351', 'EEE-485', 'CSS-464']
schedule_string = open('data.txt','r').read()
extractor = CourseScheduleExtractor(schedule_string, courses)
extractor.get_courses()
schedules = extractor.get_schedules()
#extractor.get_course_info('CSS-464')

All courses:  ['CSS-464', 'EEE-212', 'EEE-313', 'EEE-321', 'EEE-342', 'EEE-351', 'EEE-447', 'EEE-448', 'EEE-485'] 

Selected courses:  ['EEE-313', 'EEE-321', 'EEE-351', 'EEE-485', 'CSS-464'] 

Number of schedules found: 4 

EEE-313 Section: 2  Erdinç Tatar
EEE-321 Section: 1  Süleyman Serdar Kozat
EEE-351 Section: 3  Ayhan Altıntaş
EEE-485 Section: 1  Cem Tekin
CSS-464 Section: 1  Ayşegül Dündar Boral


,Mon,Tue,Wed,Thu,Fri
1,EEE-321-1,EEE-485-1,EEE-351-3,EEE-313-2,
2,EEE-321-1,EEE-485-1,EEE-351-3,EEE-313-2,
3,CSS-464-1,,,,
4,CSS-464-1,,,,
L,,,,,
5,EEE-313-2,EEE-313-2,EEE-321-1,EEE-485-1,EEE-351-3
6,EEE-313-2,EEE-313-2,EEE-321-1,EEE-485-1,EEE-351-3
7,,EEE-313-2,CSS-464-1,,EEE-321-1
8,,EEE-313-2,CSS-464-1,,EEE-321-1


EEE-313 Section: 2  Erdinç Tatar
EEE-321 Section: 1  Süleyman Serdar Kozat
EEE-351 Section: 3  Ayhan Altıntaş
EEE-485 Section: 1  Cem Tekin
CSS-464 Section: 2  Ayşegül Dündar Boral


,Mon,Tue,Wed,Thu,Fri
1,EEE-321-1,EEE-485-1,EEE-351-3,EEE-313-2,
2,EEE-321-1,EEE-485-1,EEE-351-3,EEE-313-2,
3,,CSS-464-2,,,
4,,CSS-464-2,,,
L,,,,,
5,EEE-313-2,EEE-313-2,EEE-321-1,EEE-485-1,EEE-351-3
6,EEE-313-2,EEE-313-2,EEE-321-1,EEE-485-1,EEE-351-3
7,,EEE-313-2,,CSS-464-2,EEE-321-1
8,,EEE-313-2,,CSS-464-2,EEE-321-1


EEE-313 Section: 3  Erdinç Tatar
EEE-321 Section: 1  Süleyman Serdar Kozat
EEE-351 Section: 1  Vakur Behçet Ertürk
EEE-485 Section: 1  Cem Tekin
CSS-464 Section: 1  Ayşegül Dündar Boral


,Mon,Tue,Wed,Thu,Fri
1,EEE-321-1,EEE-485-1,,EEE-351-1,EEE-313-3
2,EEE-321-1,EEE-485-1,,EEE-351-1,EEE-313-3
3,CSS-464-1,EEE-313-3,,,EEE-313-3
4,CSS-464-1,EEE-313-3,,,EEE-313-3
L,,,,,
5,EEE-351-1,,EEE-321-1,EEE-485-1,
6,EEE-351-1,,EEE-321-1,EEE-485-1,
7,,,CSS-464-1,EEE-313-3,EEE-321-1
8,,,CSS-464-1,EEE-313-3,EEE-321-1


EEE-313 Section: 3  Erdinç Tatar
EEE-321 Section: 1  Süleyman Serdar Kozat
EEE-351 Section: 3  Ayhan Altıntaş
EEE-485 Section: 1  Cem Tekin
CSS-464 Section: 1  Ayşegül Dündar Boral


,Mon,Tue,Wed,Thu,Fri
1,EEE-321-1,EEE-485-1,EEE-351-3,,EEE-313-3
2,EEE-321-1,EEE-485-1,EEE-351-3,,EEE-313-3
3,CSS-464-1,EEE-313-3,,,EEE-313-3
4,CSS-464-1,EEE-313-3,,,EEE-313-3
L,,,,,
5,,,EEE-321-1,EEE-485-1,EEE-351-3
6,,,EEE-321-1,EEE-485-1,EEE-351-3
7,,,CSS-464-1,EEE-313-3,EEE-321-1
8,,,CSS-464-1,EEE-313-3,EEE-321-1


In [ ]:
extractor.get_course_info('CSS-464')